In [1]:
import nltk
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
data=pd.read_csv('train.csv')
print("size of the data:", len(data))
data.head()

size of the data: 159571


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
# Create new features
X1 = pd.DataFrame(columns=[])
Y = pd.DataFrame(columns = ['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

# Create Features - X1
X1['length'] = data['comment_text'].apply(len)
X1['num_words'] = data['comment_text'].str.split().apply(len)
X1['caps_letter'] = data['comment_text'].apply(lambda capsL: sum(1 for c in capsL if c.isupper()))
X1['caps_word'] = data['comment_text'].apply(lambda capsW : sum(1 for c in capsW.split() if c.isupper()))
X1['exclaimations'] = data['comment_text'].apply(lambda exclaim : exclaim.count('!'))
X1['questions'] = data['comment_text'].apply(lambda questions : questions.count('?'))
X1['punct'] = data['comment_text'].apply(lambda punct: sum(punct.count(p) for p in '.,;:'))
X1['sp_char'] = data['comment_text'].apply(lambda sp_char: sum(sp_char.count(s) for s in '@#$%^*()-'))
X1['unique_words'] = data['comment_text'].apply(lambda text: len(set(w for w in text.split())))
X1['num_emoji'] = data['comment_text'].apply(lambda emoji: sum(emoji.count(w) for w in (':-)', ':)', ';-)', ';)')))
# Manipulate to find ratios
X1['capsL_to_length'] = X1['caps_letter'] / X1['length']
X1['capsW_to_length'] = X1['caps_word'] / X1['length']
X1['unique_to_words'] = X1['unique_words']/X1['num_words']



# Target values
Y['toxic'] = data ['toxic']
Y['severe_toxic'] = data['severe_toxic']
Y['obscene'] = data['obscene']
Y['threat'] = data['threat']
Y['insult'] = data['insult']
Y['identity_hate'] = data['identity_hate']


In [4]:
temp = data['comment_text']
all_comments = []
for i in range(len(temp)):
    all_comments.append(temp[i])
print('shape of all_comments= ', np.shape(all_comments))

shape of all_comments=  (159571,)


In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = 500  # cut reviews after 500 words
max_words = 1000000  # take top 1000,000 words

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(all_comments)
sequences = tokenizer.texts_to_sequences(all_comments)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

data_token = pad_sequences(sequences, maxlen=maxlen)
data_token1 = pd.DataFrame(data_token)

Found 210337 unique tokens.


In [6]:
char = keras.preprocessing.text.Tokenizer(num_words=500,char_level=True)
char.fit_on_texts(all_comments)
char_seq = char.texts_to_sequences(all_comments)
char_token = pad_sequences(char_seq, maxlen =500)

In [7]:
X2 = char_token
X3 = data_token
print(np.shape(X1))
print(np.shape(X2))
print(np.shape(X3))

(159571, 13)
(159571, 500)
(159571, 500)


In [8]:
# features1 = np.hstack([X1, X3, X2])

featrs = np.hstack([(0.1*X1), (0.6*X2), (0.3*X3)])
labels = Y

from sklearn.cross_validation import train_test_split
Train_text3, Test_text3, Train_label3, Test_label3 = train_test_split(featrs, labels, test_size=0.05, random_state=42)

C:\Users\ghoshal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from keras.layers import SimpleRNN
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Activation

model3 = Sequential()
model3.add(Embedding(max_words, 32))
model3.add(LSTM(32))
model3.add(Dense(6, activation='sigmoid'))
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
fit_lstm3 = model3.fit(Train_text3, Train_label3,epochs=10,batch_size=128, validation_data= (Test_text3, Test_label3))

Train on 151592 samples, validate on 7979 samples
Epoch 1/10
151592/151592 [==============================] - 1426s 9ms/step - loss: 0.1170 - acc: 0.9660 - val_loss: 0.0772 - val_acc: 0.9750
Epoch 2/10
151592/151592 [==============================] - 1395s 9ms/step - loss: 0.0710 - acc: 0.9756 - val_loss: 0.0714 - val_acc: 0.9754
Epoch 3/10
151592/151592 [==============================] - 1397s 9ms/step - loss: 0.0622 - acc: 0.9777 - val_loss: 0.0735 - val_acc: 0.9745
Epoch 4/10
151592/151592 [==============================] - 1391s 9ms/step - loss: 0.0574 - acc: 0.9792 - val_loss: 0.0724 - val_acc: 0.9756
Epoch 5/10
151592/151592 [==============================] - 1398s 9ms/step - loss: 0.0622 - acc: 0.9774 - val_loss: 0.0723 - val_acc: 0.9758
Epoch 6/10
151592/151592 [==============================] - 1391s 9ms/step - loss: 0.0517 - acc: 0.9810 - val_loss: 0.0738 - val_acc: 0.9758
Epoch 7/10
151592/151592 [==============================] - 1390s 9ms/step - loss: 0.0480 - acc: 0.9823 

## Test data

In [12]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [11]:
# Create X1
X1_test = pd.DataFrame(columns=[])

# Create Features - X1
X1_test['length'] = test_data['comment_text'].apply(len)
X1_test['num_words'] = test_data['comment_text'].str.split().apply(len)
X1_test['caps_letter'] = test_data['comment_text'].apply(lambda capsL: sum(1 for c in capsL if c.isupper()))
X1_test['caps_word'] = test_data['comment_text'].apply(lambda capsW : sum(1 for c in capsW.split() if c.isupper()))
X1_test['exclaimations'] = test_data['comment_text'].apply(lambda exclaim : exclaim.count('!'))
X1_test['questions'] = test_data['comment_text'].apply(lambda questions : questions.count('?'))
X1_test['punct'] = test_data['comment_text'].apply(lambda punct: sum(punct.count(p) for p in '.,;:'))
X1_test['sp_char'] = test_data['comment_text'].apply(lambda sp_char: sum(sp_char.count(s) for s in '@#$%^*()-'))
X1_test['unique_words'] = test_data['comment_text'].apply(lambda text: len(set(w for w in text.split())))
X1_test['num_emoji'] = test_data['comment_text'].apply(lambda emoji: sum(emoji.count(w) for w in (':-)', ':)', ';-)', ';)')))
# Manipulate to find ratios
X1_test['capsL_to_length'] = X1_test['caps_letter'] / X1['length']
X1_test['capsW_to_length'] = X1_test['caps_word'] / X1['length']
X1_test['unique_to_words'] = X1_test['unique_words']/X1['num_words']


In [13]:
temp_test = test_data['comment_text']
test_comments = []
for i in range(len(temp_test)):
    test_comments.append(temp_test[i])
print('shape of all_comments= ', np.shape(test_comments))

shape of all_comments=  (153164,)


In [19]:
# X2

maxlen = 500  # cut reviews after 500 words
max_words = 1000000  # take top 1000,000 words

tokenizer_test = Tokenizer(num_words=max_words)
tokenizer_test.fit_on_texts(test_comments)
sequences_test = tokenizer_test.texts_to_sequences(test_comments)
word_index_test = tokenizer_test.word_index

print('Found %s unique tokens.' % len(word_index_test))

test_token = pad_sequences(sequences_test, maxlen=maxlen)
test_token1 = pd.DataFrame(data_token)

Found 273192 unique tokens.


In [15]:
char_test = keras.preprocessing.text.Tokenizer(num_words=500,char_level=True)
char_test.fit_on_texts(test_comments)
char_seq_test = char_test.texts_to_sequences(test_comments)
char_token_test = pad_sequences(char_seq_test, maxlen =500)

In [20]:
X2_test = test_token
X3_test = char_token_test
print(np.shape(X1_test), np.shape(X2_test), np.shape(X3_test))

(153164, 13) (153164, 500) (153164, 500)


In [22]:
TEST = np.hstack([(0.1*X1_test), (0.6*X2_test), (0.3*X3_test)])
predict = model3.predict(TEST)

In [44]:
index = test_data[test_data.columns[0]]
index = np.reshape(index, (len(test_data),1))

C:\Users\ghoshal\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [61]:
answer = np.hstack([index, predict])

In [62]:
colnames = ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
answer = pd.DataFrame(answer, columns=colnames)

In [64]:
answer.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.020571,0.000136871,0.000994945,0.000561548,0.00140842,0.000714796
1,0000247867823ef7,0.0416148,0.000145815,0.00310565,0.000449397,0.00289282,0.000561309
2,00013b17ad220c46,0.00872038,8.26892e-05,0.00085414,0.000269422,0.00079267,0.000255726
3,00017563c3f7919a,0.0360134,0.000160344,0.00267686,0.000523436,0.0027998,0.000674375
4,00017695ad8997eb,0.0166309,9.96004e-05,0.00116931,0.000312351,0.00128385,0.000350295


In [77]:
answer.to_csv('Submission_Gourav.csv')